This exercise will require you to pull some data from https://data.nasdaq.com/ (formerly Quandl API).

As a first step, you will need to register a free account on the https://data.nasdaq.com/ website.

After you register, you will be provided with a unique API key, that you should store:

*Note*: Use a `.env` file and put your key in there and `python-dotenv` to access it in this notebook. 

The code below uses a key that was used when generating this project but has since been deleted. Never submit your keys to source control. There is a `.env-example` file in this repository to illusrtate what you need. Copy that to a file called `.env` and use your own api key in that `.env` file. Make sure you also have a `.gitignore` file with a line for `.env` added to it. 

The standard Python gitignore is [here](https://github.com/github/gitignore/blob/master/Python.gitignore) you can just copy that. 

In [ ]:
# get api key from your .env file
import os
from dotenv import load_dotenv

load_dotenv()
API_KEY = os.getenv('NASDAQ_API_KEY')

print(API_KEY)

Nasdaq Data has a large number of data sources, but, unfortunately, most of them require a Premium subscription. Still, there are also a good number of free datasets.

For this mini project, we will focus on equities data from the Frankfurt Stock Exhange (FSE), which is available for free. We'll try and analyze the stock prices of a company called Carl Zeiss Meditec, which manufactures tools for eye examinations, as well as medical lasers for laser eye surgery: https://www.zeiss.com/meditec/int/home.html. The company is listed under the stock ticker AFX_X.

You can find the detailed Nasdaq Data API instructions here: https://docs.data.nasdaq.com/docs/in-depth-usage

While there is a dedicated Python package for connecting to the Nasdaq API, we would prefer that you use the *requests* package, which can be easily downloaded using *pip* or *conda*. You can find the documentation for the package here: http://docs.python-requests.org/en/master/ 

Finally, apart from the *requests* package, you are encouraged to not use any third party Python packages, such as *pandas*, and instead focus on what's available in the Python Standard Library (the *collections* module might come in handy: https://pymotw.com/3/collections/).
Also, since you won't have access to DataFrames, you are encouraged to us Python's native data structures - preferably dictionaries, though some questions can also be answered using lists.
You can read more on these data structures here: https://docs.python.org/3/tutorial/datastructures.html

Keep in mind that the JSON responses you will be getting from the API map almost one-to-one to Python's dictionaries. Unfortunately, they can be very nested, so make sure you read up on indexing dictionaries in the documentation provided above.

In [9]:
# First, import the relevant modules
import requests



Note: API's can change a bit with each version, for this exercise it is reccomended to use the nasdaq api at `https://data.nasdaq.com/api/v3/`. This is the same api as what used to be quandl so `https://www.quandl.com/api/v3/` should work too.

Hint: We are looking for the `AFX_X` data on the `datasets/FSE/` dataset.

In [31]:
# Now, call the Nasdaq API and pull out a small sample of the data (only one day) to get a glimpse
# into the JSON structure that will be returne
url = "https://data.nasdaq.com/api/v3/datasets/FSE/AFX_X"
params = {
          "api_key": API_KEY
        }
r = requests.get(url, params=params)

In [ ]:
# Inspect the JSON structure of the object you created, and take note of how nested it is,
# as well as the overall structure
r.json()

These are your tasks for this mini project:

1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).
2. Convert the returned JSON object into a Python dictionary.
3. Calculate what the highest and lowest opening prices were for the stock in this period.
4. What was the largest change in any one day (based on High and Low price)?
5. What was the largest change between any two days (based on Closing Price)?
6. What was the average daily trading volume during this year?
7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

In [40]:
#1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).
params = {
    "start_date":"2017-01-01",
    "end_date":"2017-12-31",
    "api_key":API_KEY
}
response = requests.get(url, params=params)

In [ ]:
#2. Convert the returned JSON object into a Python dictionary.
data_2017 = response.json()
data_2017

In [50]:
data_2017.keys()

dict_keys(['dataset'])

In [52]:
data_2017['dataset'].keys()

dict_keys(['id', 'dataset_code', 'database_code', 'name', 'description', 'refreshed_at', 'newest_available_date', 'oldest_available_date', 'column_names', 'frequency', 'type', 'premium', 'limit', 'transform', 'column_index', 'start_date', 'end_date', 'data', 'collapse', 'order', 'database_id'])

In [81]:
#Here we see a list of a list holding a bunch of data for a certain date, so in order to find out what it is, in the next cell we will find out what the names for each column will
#be
data_2017_list = data_2017['dataset']['data']
#Here we see the 2nd columns is the Open price, 5th is the closing price, We can now find the highest opening and lowest prices now. 
print(data_2017['dataset']['column_names'])

['Date', 'Open', 'High', 'Low', 'Close', 'Change', 'Traded Volume', 'Turnover', 'Last Price of the Day', 'Daily Traded Units', 'Daily Turnover']


In [80]:
#3. 
max_p = float('-inf')
min_p = float('inf')
for i in data_2017_list:
    if i[1] != None:
        max_p = max(max_p, i[1])
        min_p = min(min_p, i[1])

print(f"The highest opening price for AFX_X in the year 2017 was : ${max_p}")
print(f"The lowest opening price for AFX_X in the year 2017 was: ${min_p}")

The highest opening price for AFX_X in the year 2017 was : $53.11
The lowest opening price for AFX_X in the year 2017 was: $34.0


In [85]:
#4
greatest_change = float('-inf')
for i in data_2017_list:
    if (i[2] and i[3]) != None:
        greatest_change = max(greatest_change, (i[2] - i[3]))
print(f"The greatest change in one day for AFX_X in the year 2017 was: ${greatest_change:.2f}")

The greatest change in one day for AFX_X in the year 2017 was: $2.81


In [89]:
#5
largets_two_day_close = float('-inf')
previous_day_close = None
for i in data_2017_list:
    if previous_day_close is not None and i[4] is not None:
        change = abs(i[4] - previous_day_close)
        largets_two_day_close = max(largets_two_day_close, change)
    previous_day_close = i[4]
print(f"The greatest change in two days for AFX_X in the year 2017 was {largets_two_day_close:.2f}")

The greatest change in two days for AFX_X in the year 2017 was 2.56


In [90]:
#6
tradingvolume_avg = 0
counter=0
for i in data_2017_list:
    if i[6] != None:
        tradingvolume_avg+=i[6]
        counter+=1
print(f"The average daily trading volume during this year for AFX_X was {tradingvolume_avg/counter:.2f}")


The average daily trading volume during this year for AFX_X was 89124.34


In [93]:
#7
l = []
for i in data_2017_list:
    if i[6] != None:
        l.append(i[6])
l.sort()
print(f"The median of the trading volume is for AFX_X in the yera 2017 is: {l[len(l)//2]}")

The median of the trading volume is for AFX_X in the yera 2017 is: 76286.0
